In [3]:
cd ../../src

/home/divye/Documents/Projects/mic_predictor/src


In [4]:
import biovec
import numpy as np
import pandas as pd
from itertools import chain, combinations
from collections import Counter

from utils import *

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [5]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

In [6]:
avp_ic50 = pd.read_csv("../data/raw/AVP-IC50Pred_train.csv")
ha_avp = pd.read_csv("../data/raw/HA_AVP.csv")

In [7]:
df = pd.concat([avp_ic50[['Sequence','MIC']], ha_avp], axis=0).drop_duplicates(['Sequence']).reset_index(drop=True)
df = sequence_filtering(df)

In [9]:
dipep_freq = reduce_by_kmer_frequency(df, kmer=2)

In [10]:
uniprot_embedding = biovec.models.load_protvec("../data/embeddings/uniprot__kmer_3_contextWindow_10_vector_100_reduction_None")

avg_protvec = convert_sequences_to_avg_vectors(df['Sequence'], uniprot_embedding, kmer=5)
avg_protvec = avg_protvec.reset_index(drop=True)

Creating vectors: 100%|██████████| 712/712 [00:03<00:00, 195.51sequence/s]


In [11]:
X = pd.concat([dipep_freq, avg_protvec], axis=1)

In [12]:
y = df['MIC']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
knn = KNeighborsRegressor(metric='euclidean', n_neighbors=1, weights='uniform')

In [15]:
_ = knn.fit(X_train, y_train)

In [16]:
y_pred = knn.predict(X_test)

In [17]:
mean_absolute_percentage_error(y_test, y_pred)

4536851.345377963